In [71]:
import pandas as pd

In [72]:
df = pd.read_csv("resources.csv")
df

,Resources,Category
0,https://www.epa.gov/watersense/fix-leak-week\n,Water
1,https://www.epa.gov/watersense/statistics-and-...,Water
2,https://www.nidwater.com/5-facts-you-need-to-k...,Water
3,https://parade.com/1192402/jessicasager/earth-...,History
4,https://www.intermountainhistories.org/tours/s...,History
...,...,...
101,https://sitn.hms.harvard.edu/uncategorized/201...,Water
102,https://astrobiology.nasa.gov/education/alp/wa...,Water
103,https://www.maine.gov/dacf/php/gotpests/otherc...,WIldlife
104,https://www.nmhealth.org/publication/view/guid...,WIldlife


In [73]:
# remove the trailing spaces or '\n' from df
df.replace('\n', '', regex=True, inplace=True)
df.replace(' ', '', regex=True, inplace=True)
df

,Resources,Category
0,https://www.epa.gov/watersense/fix-leak-week,Water
1,https://www.epa.gov/watersense/statistics-and-...,Water
2,https://www.nidwater.com/5-facts-you-need-to-k...,Water
3,https://parade.com/1192402/jessicasager/earth-...,History
4,https://www.intermountainhistories.org/tours/s...,History
...,...,...
101,https://sitn.hms.harvard.edu/uncategorized/201...,Water
102,https://astrobiology.nasa.gov/education/alp/wa...,Water
103,https://www.maine.gov/dacf/php/gotpests/otherc...,WIldlife
104,https://www.nmhealth.org/publication/view/guid...,WIldlife


In [74]:
# Pick a random row from df
df.sample(1)['Resources'].iloc[0]

'https://www.un.org/esa/dsd/csd/csd_aboucsd.shtml'

In [75]:
# pick a randow row where the column 'Category' is 'Water' in df
len(df.loc[df['Category'] == 'Water'])
# df.loc[df['Category'] == 'Energy'].sample(1)['Resources'].iloc[0]


6

In [76]:
import nest_asyncio
from langchain_community.document_loaders import AsyncChromiumLoader

nest_asyncio.apply()


url = df.sample(1)['Resources'].iloc[0]
loader = AsyncChromiumLoader([url])
docs = loader.load()


In [77]:
url

'https://www.epa.gov/watersense/fix-leak-week'

In [78]:
from langchain_community.document_transformers import BeautifulSoupTransformer

bs_transformer = BeautifulSoupTransformer()
docs_transformed1 = bs_transformer.transform_documents(docs,       
        unwanted_tags=["script", "style", "footer", "header", "nav", "form", "noscript"],
        remove_lines=True,
        remove_comments=True)


In [79]:
print(docs_transformed1[0].page_content)

Skip to main content (#main)           Official websites use .gov  A .gov website belongs to an official government organization in the United States.        Secure .gov websites use HTTPS  A lock ( Lock A locked padlock ) or https:// means you’ve safely connected to the .gov website. Share sensitive information only on official, secure websites.               WaterSense    Contact Us (/watersense/forms/contact-us-about-watersense)       Fix a Leak Week         Leaks Can Run, but They Can't Hide  Are you ready to chase down leaks? Household leaks can waste nearly 1 trillion gallons of water annually nationwide, so each year we hunt down the drips during Fix a Leak Week. Mark your calendars for EPA's annual Fix a Leak Week, March 18 through 24, 2024 —but remember that you can find and fix leaks inside and outside your home to save valuable water and money all year long.  From family fun runs to leak detection contests to WaterSense demonstrations, Fix a Leak Week events happen from coas

In [80]:
docs_transformed1[0]

Document(page_content='Skip to main content (#main)           Official websites use .gov  A .gov website belongs to an official government organization in the United States.        Secure .gov websites use HTTPS  A lock ( Lock A locked padlock ) or https:// means you’ve safely connected to the .gov website. Share sensitive information only on official, secure websites.               WaterSense    Contact Us (/watersense/forms/contact-us-about-watersense)       Fix a Leak Week         Leaks Can Run, but They Can\'t Hide  Are you ready to chase down leaks? Household leaks can waste nearly\xa01 trillion gallons of water annually nationwide, so each year we hunt down the drips during Fix a Leak Week. Mark your calendars for EPA\'s annual Fix a Leak Week, March 18 through 24, 2024 —but remember that you can find and fix leaks inside and outside your home to save valuable water and money all year long.  From family fun runs to leak detection contests to WaterSense demonstrations, Fix a Leak 

In [81]:
formatted = [
        f"Source ID: {i}\nArticle Title: {doc.metadata}\nArticle Snippet: {doc.page_content}"
        for i, doc in enumerate(docs_transformed1)
    ]
formatted

['Source ID: 0\nArticle Title: {\'source\': \'https://www.epa.gov/watersense/fix-leak-week\'}\nArticle Snippet: Skip to main content (#main)           Official websites use .gov  A .gov website belongs to an official government organization in the United States.        Secure .gov websites use HTTPS  A lock ( Lock A locked padlock ) or https:// means you’ve safely connected to the .gov website. Share sensitive information only on official, secure websites.               WaterSense    Contact Us (/watersense/forms/contact-us-about-watersense)       Fix a Leak Week         Leaks Can Run, but They Can\'t Hide  Are you ready to chase down leaks? Household leaks can waste nearly\xa01 trillion gallons of water annually nationwide, so each year we hunt down the drips during Fix a Leak Week. Mark your calendars for EPA\'s annual Fix a Leak Week, March 18 through 24, 2024 —but remember that you can find and fix leaks inside and outside your home to save valuable water and money all year long.  

In [86]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, create_citation_fuzzy_match_chain
from langchain_core.prompts import PromptTemplate
from trivia import create_trivia_with_citation__chain

import dotenv

dotenv.load_dotenv()


True

In [87]:
# Define the model
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# Create the chain
llm_chain = create_trivia_with_citation__chain(llm)

# Use the model
question = "4"
result = llm_chain.run(question=question, context=formatted)

In [88]:
result

QuestionAnswer(question='What is the average amount of water wasted by the leaks in the average household per year?', wrong_answer='100 gallons', difficulty='Medium', answer=[FactWithEvidence(fact="The average household's leaks can account for nearly 10,000 gallons of water wasted every year.", trivia_answer='10,000 gallons', substring_quote=["The average household's leaks can account for nearly 10,000 gallons of water wasted every year."], sources=['https://www.epa.gov/watersense/fix-leak-week'])])

In [68]:
result.answer


[FactWithEvidence(fact='Earth Day, an event to increase public awareness of the world’s environmental problems, is celebrated in the United States for the first time on April 22, 1970.', trivia_answer='April 22, 1970', substring_quote=['Earth Day, an event to increase public awareness of the world’s environmental problems, is celebrated in the United States for the first time on April 22, 1970.'], sources=['https://www.history.com/this-day-in-history/the-first-earth-day'])]